In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, OpenAI, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, ConfigurableField

In [2]:
loader = PyPDFLoader("Ancient_History_Revision.pdf")
documents = loader.load_and_split()

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

In [4]:
retriever = vectorstore.as_retriever()

In [5]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [6]:
prompt = ChatPromptTemplate.from_template(template)
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
openai = OpenAI(model="gpt-3.5-turbo-instruct")
anthropic = ChatAnthropic(model="claude-2")
model = (
    chat_openai
    .with_fallbacks([anthropic])
    .configurable_alternatives(
        ConfigurableField(id="model"),
        default_key="chat_openai",
        openai=openai,
        anthropic=anthropic,
    )
)
output_parser = StrOutputParser()

In [7]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [8]:
chain = setup_and_retrieval | prompt | model | output_parser

In [9]:
chain.invoke("How harsha is important. Please explain in detail")

"Harsha, also known as Harshavardhana, was an important ruler in Indian history during the transition from ancient to medieval times. His reign marked a shift towards feudal and decentralized administration, different from the centralized rule of the Guptas. Harsha's kingdom was characterized by significant military power, with a large army consisting of 100,000 horses and 60,000 elephants.\n\nOne of the key aspects of Harsha's reign was his unique approach to governance and revenue allocation. According to the Chinese pilgrim Hsuan Tsang, Harsha divided his revenues into four parts - one for the king's expenditure, another for scholars, a third for officials and public servants, and a fourth for religious purposes. This system reflected a balance between secular and religious interests in the administration.\n\nHarsha's court and life during his reign were vividly described by Hsuan Tsang, providing valuable insights into the social, economic, and religious aspects of that period. The